<a href="https://colab.research.google.com/github/sayakju/llm-practice/blob/main/basic_cc_statement_analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install --quiet google-api-python-client pdfplumber PyPDF2 transformers huggingface_hub

In [7]:
import os
import base64
import pdfplumber
import PyPDF2
from google.colab import files
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from transformers import pipeline

# === Upload OAuth files ===
print("Please upload both 'token.json' and 'credentials.json'")
uploaded = files.upload()  # This will prompt file upload

# Save uploaded files to disk
for fn in uploaded.keys():
    with open(fn, 'wb') as f:
        f.write(uploaded[fn])

# Confirm upload
if not os.path.exists("token.json") or not os.path.exists("credentials.json"):
    raise FileNotFoundError("Both 'token.json' and 'credentials.json' must be uploaded.")
else:
    print("✅ Files uploaded successfully.")

# CONFIG
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
CREDENTIALS_PATH = 'token.json'
PDF_PASSWORD = input("Enter your PDF password: ")

Please upload both 'token.json' and 'credentials.json'


Saving token.json to token (5).json
Saving credentials.json to credentials (5).json
✅ Files uploaded successfully.
Enter your PDF password: SAYA1307


In [8]:
# Initialize Gmail API
creds = Credentials.from_authorized_user_file(CREDENTIALS_PATH, SCOPES)
gmail_service = build('gmail', 'v1', credentials=creds)

# Load summarization pipeline from Hugging Face
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

Device set to use cpu


In [9]:
def search_emails(query):
    results = gmail_service.users().messages().list(userId='me', q=query).execute()
    return results.get('messages', [])

def get_email_attachments(msg_id):
    msg = gmail_service.users().messages().get(userId='me', id=msg_id, format='full').execute()
    attachments = []
    def extract(parts):
        for part in parts:
            filename = part.get("filename", "")
            body = part.get("body", {})
            if filename.endswith(".PDF") or filename.endswith(".pdf"):
                if "attachmentId" in body:
                    att_id = body["attachmentId"]
                    attachment = gmail_service.users().messages().attachments().get(userId='me', messageId=msg_id, id=att_id).execute()
                    data = base64.urlsafe_b64decode(attachment['data'])
                    attachments.append((filename, data))
            if "parts" in part:
                extract(part['parts'])
    payload = msg.get("payload", {})
    if "parts" in payload:
        extract(payload['parts'])
    return attachments

In [10]:
def extract_text_from_pdf(data):
    with open("temp.pdf", "wb") as f:
        f.write(data)
    try:
        with open("temp.pdf", "rb") as f:
            reader = PyPDF2.PdfReader(f)
            if reader.is_encrypted:
                reader.decrypt(PDF_PASSWORD)
            text = ""
            for page in reader.pages:
                text += page.extract_text() or ""
        return text
    except Exception as e:
        print("PDF decryption or extraction failed:", e)
        return ""

In [11]:
pip install transformers torch


In [12]:
from transformers import pipeline, AutoTokenizer

# Load summarizer and tokenizer
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")

# Main workflow
query = 'subject:"Your HDFC Bank - Infinia Credit Card Statement" has:attachment'
messages = search_emails(query)
print(f"Found {len(messages)} email(s) matching the query.")

for msg in messages:
    print(f"Processing email ID: {msg['id']}")
    attachments = get_email_attachments(msg['id'])
    for fname, data in attachments:
        print(f"  - {fname}")
        text = extract_text_from_pdf(data)
        if text:
            # Tokenize and truncate to model's max input length (1024 tokens)
            tokens = tokenizer.encode(text, truncation=True, max_length=1024)
            short_text = tokenizer.decode(tokens, skip_special_tokens=True)

            print("Summarizing extracted text...")
            summary = summarizer(short_text, max_length=512, min_length=100, do_sample=False)
            print("Summary:", summary[0]['summary_text'])


Device set to use cpu


Found 21 email(s) matching the query.
Processing email ID: 197f1e569d3ad3af
  - 4375XXXXXXXXXX76_09-07-2025.PDF
Summarizing extracted text...
Summary: HDFC Bank Credit Card is available for purchase in India. The card is issued by the HDFC Bank Cards Division, Door No 94 SP, Estate BusStand, Wavin Main Road, Mogappair West, Chennai 600058. The bank will report you if you do not pay the minimum due amount within three days of the due date. For more information, please visit: http://www.hfc.com/cards/credit-card/card-details.php?cid=14,21,000 13,26,993 5,68,400.
Processing email ID: 197577c42cd18317
  - 4375XXXXXXXXXX76_09-06-2025.PDF
Summarizing extracted text...
Summary: HDFC Bank Credit Card is available for purchase in India. The card has been issued by the HDFC Bank Cards Division, Door No 94 SP, Estate BusStand, Wavin Main Road, Mogappair West, Chennai 600058. The account number is: 4375 46XX XXXX 0176. The bank will report you if you do not pay the minimum due amount within three 

KeyboardInterrupt: 